# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
import pandas as pd
import requests

# 1) seu código aqui
sinasc = pd.read_csv('SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape
# Não há duplicados

(27028, 69)


(27028, 69)

In [2]:
# 2) seu código aqui
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [3]:
# 3) seu código aqui
sinasc_interesse = sinasc[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
sinasc_interesse.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [4]:
# 4) seu código aqui
sinasc_interesse['APGAR5'].shape

(27028,)

In [5]:
sinasc_interesse['APGAR5'].isna().sum()

103

In [6]:
sinasc_APGAR5 = sinasc_interesse['APGAR5'].dropna()
sinasc_APGAR5.isna().sum()

0

In [7]:
# 5) seu código aqui
sinasc_interesse['ESTCIVMAE'].isna().sum()

317

In [8]:
sinasc_interesse['CONSULTAS'].isna().sum()

0

In [9]:
sinasc_ESTCIVMAE = sinasc_interesse['ESTCIVMAE'].fillna(9)
sinasc_ESTCIVMAE.isna().sum()

0

In [10]:
# 6) Seu código aqui
sinasc_QTDFILVIVO = sinasc_interesse['QTDFILVIVO'].fillna(0)
sinasc_QTDFILVIVO.isna().sum()

0

In [11]:
# 7) seu código aqui ESCMAE, GESTACAO, GRAVIDEZ
sinasc_interesse['ESCMAE'].value_counts()


ESCMAE
8 a 11 anos        15633
12 anos ou mais     5767
4 a 7 anos          4760
1 a 3 anos           516
Nenhuma               40
Name: count, dtype: int64

Os dados não preenchidos da escolaridade da mãe foram substituidos pelo número 9, que significa 'ignorado':

In [12]:
sinasc_ESCMAE = sinasc_interesse['ESCMAE'].fillna(9)
sinasc_ESCMAE.isna().sum()

0

Os dados não preenchidos da gestação e gravidez foram substituidos pelo valor que aparece com a maior frequência, por serem considerados o "normal":

In [13]:
sinasc_interesse['GESTACAO'].isna().sum()

1232

In [14]:
sinasc_interesse['GESTACAO'].value_counts()

GESTACAO
37 a 41 semanas        22589
32 a 36 semanas         2249
42 semanas e mais        700
28 a 31 semanas          165
22 a 27 semanas           86
Menos de 22 semanas        7
Name: count, dtype: int64

In [15]:
sinasc_GESTACAO = sinasc_interesse['GESTACAO'].fillna('37 a 41 semanas')
sinasc_GESTACAO.isna().sum()

0

In [16]:
sinasc_interesse['GRAVIDEZ'].isna().sum()

79

In [17]:
sinasc_interesse['GRAVIDEZ'].value_counts()

GRAVIDEZ
Única              26452
Dupla                492
Tríplice e mais        5
Name: count, dtype: int64

In [18]:
sinasc_GRAVIDEZ = sinasc_interesse['GRAVIDEZ'].fillna('Única')
sinasc_GRAVIDEZ.isna().sum()

0

In [19]:
# 8) seu código aqui
sinasc_APGAR5 = sinasc_interesse['APGAR5'].dropna()
sinasc_APGAR5 = pd.DataFrame(sinasc_APGAR5)
sinasc_APGAR5

,APGAR5
0,10.0
1,9.0
2,10.0
3,10.0
4,10.0
...,...
27023,9.0
27024,9.0
27025,10.0
27026,9.0


In [20]:
# Entre 8 e 10 está em uma faixa 'normal'.
sinasc_APGAR5.loc[sinasc_APGAR5['APGAR5'] >= 8, 'classificação_asfixia'] = 'normal'
# Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'.
sinasc_APGAR5.loc[(sinasc_APGAR5['APGAR5'] >=6) & (sinasc_APGAR5['APGAR5'] <= 7), 'classificação_asfixia'] = 'asfixia leve'
# Entre 4 e 5 significa 'asfixia moderada'.
sinasc_APGAR5.loc[(sinasc_APGAR5['APGAR5'] >= 4) & (sinasc_APGAR5['APGAR5'] <= 5), 'classificação_asfixia'] = 'asfixia moderada'
# Entre 0 e 3 significa 'asfixia severa'.
sinasc_APGAR5.loc[(sinasc_APGAR5['APGAR5'] <= 3), 'classificação_asfixia'] = 'asfixia severa'

In [21]:
sinasc_APGAR5['classificação_asfixia'].value_counts()

classificação_asfixia
normal              26463
asfixia leve          320
asfixia severa         74
asfixia moderada       68
Name: count, dtype: int64

In [22]:
# 9) seu código aqui
sinasc_APGAR5['classificação_asfixia'] = sinasc_APGAR5['classificação_asfixia'].str.replace(' ', '_').str.lower()

In [23]:
sinasc_APGAR5['classificação_asfixia'].value_counts()

classificação_asfixia
normal              26463
asfixia_leve          320
asfixia_severa         74
asfixia_moderada       68
Name: count, dtype: int64